<a href="https://colab.research.google.com/github/Jaehwi-So/DeepLearning_Study/blob/main/DL06_NLP_RNN%EA%B8%B0%EB%B0%98_%EC%9E%90%EC%97%B0%EC%96%B4%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('drive/MyDrive/DL2024_201810776/week11/')

%load_ext autoreload
%autoreload 2

In [17]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, LSTM, GRU, Dense
import matplotlib.pyplot as plt

# A. 영화 리뷰 학습 후 문장 긍정/부정 구분하기

In [4]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding

## 1. 데이터셋 로드
IMDB(Internet Movie Database)는 영화 정보와 리뷰 등을 포함하는 온라인 데이터베이스이다

In [5]:
max_features = 10000  # 단어 사전 : 사용할 단어의 최대 개수 (Baby, Panda..)
max_len = 500  # 패딩할 시퀀스의 최대 길이 (비어있는 부분을 0으로 채워줌)

(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words=max_features)

17464789/17464789 [==============================] - 0s 0us/step
[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 14, 20, 56, 33, 2401, 18, 457, 88, 13, 2626, 1400, 45, 3171, 13, 70, 79, 49, 706, 919, 13, 16, 355, 340, 355, 1696, 96, 143, 4, 22, 32, 289, 7, 61, 369, 71, 2359, 5, 13, 16, 131, 2073, 249, 114, 249, 229, 249, 20, 13, 28, 126, 110, 13, 473, 8, 569, 61, 419, 56, 429, 6, 1513, 18, 35, 534, 95, 474, 570, 5, 25, 124, 138, 88, 12, 421, 1543, 52, 725, 6397, 61, 419, 11, 13, 1571, 15, 1543, 20, 11, 4, 2, 5, 296, 12, 3524, 5, 15, 421, 128, 74, 233, 334, 207, 126, 224, 12, 562, 298, 2167, 1272, 7, 2601, 5, 516, 988, 43, 8, 79, 120, 15, 595, 13, 784, 25, 3171, 18, 165, 170, 143, 19, 14, 5, 7224, 6, 226, 251, 7, 61, 113]


In [13]:
from utils import decode_imdb, sentence_to_vector

# 4번째 리뷰 확인
print(X_train[4]) # 숫자 하나하나가 단어, 숫자가 높을수록 빈도가 높음
print(len(X_train[4])) #147개의 단어
decoded_review = decode_imdb(X_train[4])
print(decoded_review) # 번역된 리뷰

[1, 249, 1323, 7, 61, 113, 10, 10, 13, 1637, 14, 20, 56, 33, 2401, 18, 457, 88, 13, 2626, 1400, 45, 3171, 13, 70, 79, 49, 706, 919, 13, 16, 355, 340, 355, 1696, 96, 143, 4, 22, 32, 289, 7, 61, 369, 71, 2359, 5, 13, 16, 131, 2073, 249, 114, 249, 229, 249, 20, 13, 28, 126, 110, 13, 473, 8, 569, 61, 419, 56, 429, 6, 1513, 18, 35, 534, 95, 474, 570, 5, 25, 124, 138, 88, 12, 421, 1543, 52, 725, 6397, 61, 419, 11, 13, 1571, 15, 1543, 20, 11, 4, 2, 5, 296, 12, 3524, 5, 15, 421, 128, 74, 233, 334, 207, 126, 224, 12, 562, 298, 2167, 1272, 7, 2601, 5, 516, 988, 43, 8, 79, 120, 15, 595, 13, 784, 25, 3171, 18, 165, 170, 143, 19, 14, 5, 7224, 6, 226, 251, 7, 61, 113]
147
the sure themes br only acting i i was favourite as on she they hat but already most was scares minor if flash was well also good 8 older was with enjoy used enjoy phone too i'm of you an job br only women than robot to was with these unexpected sure little sure guy sure on was one your life was children in particularly only yes sh

# 2. 데이터 전처리
- 학습시킬 벡터의 길이(max_len)은 500이다. 따라서 벡터의 길이를 맞춰주어야 한다.
- 시퀀스 패딩 : 만약 147개의 Word vector가 있다면 353개의 Padding을 추가해준다.
- 뒷부분이 영향력이 크므로 앞부분에 패딩처리를 한다.

In [14]:
X_train = pad_sequences(X_train, maxlen=max_len)
X_test = pad_sequences(X_test, maxlen=max_len)

In [15]:
X_train[4]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# 3. 모델 생성
1. 임베딩 벡터 input_dim=max_features, output_dim = 128, input_length=max_len
2. LSTM hidden layer 64개
3. Dense 출력 1개, activation은 sigmoid

In [18]:
nlp_model = Sequential([
    Embedding(input_dim=max_features, output_dim=128, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

nlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nlp_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 500, 128)          1280000   
                                                                 
 lstm (LSTM)                 (None, 64)                49408     
                                                                 
 dense (Dense)               (None, 1)                 65        
                                                                 
Total params: 1329473 (5.07 MB)
Trainable params: 1329473 (5.07 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 4. 모델 학습

In [20]:
nlp_history = nlp_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
313/313 [==============================] - 232s 731ms/step - loss: 0.5293 - accuracy: 0.7524 - val_loss: 0.4243 - val_accuracy: 0.8102
Epoch 2/5
313/313 [==============================] - 236s 753ms/step - loss: 0.3150 - accuracy: 0.8734 - val_loss: 0.3350 - val_accuracy: 0.8684
Epoch 3/5
313/313 [==============================] - 234s 747ms/step - loss: 0.2127 - accuracy: 0.9208 - val_loss: 0.3281 - val_accuracy: 0.8588
Epoch 4/5
313/313 [==============================] - 226s 720ms/step - loss: 0.1503 - accuracy: 0.9477 - val_loss: 0.3486 - val_accuracy: 0.8562
Epoch 5/5
313/313 [==============================] - 226s 722ms/step - loss: 0.1109 - accuracy: 0.9620 - val_loss: 0.3797 - val_accuracy: 0.8656


# 5. 모델 평가 및 예측

In [21]:
test_loss, test_acc = nlp_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

782/782 [==============================] - 67s 86ms/step - loss: 0.4082 - accuracy: 0.8573
Test Accuracy: 0.8573


In [32]:
encode_test_sentence = sentence_to_vector('like this actor')
encode_test_sentence = pad_sequences(encode_test_sentence, maxlen=max_len)

nlp_model.predict(encode_test_sentence) #긍정/부정 판단하기 (수치가 낮을수록 부정)

[37, 11, 281]
1/1 [==============================] - 0s 51ms/step


array([[0.5339742]], dtype=float32)

In [25]:
encode_test_sentence = sentence_to_vector('hate the movie')
encode_test_sentence = pad_sequences(encode_test_sentence, maxlen=max_len)

nlp_model.predict(encode_test_sentence) #긍정/부정 판단하기 (수치가 낮을수록 부정)

[781, 1, 17]
1/1 [==============================] - 0s 48ms/step


array([[0.22174719]], dtype=float32)

<br><hr><hr><br>
# B. 네이버 영화 리뷰(한글) 학습 후 문장의 긍정/부정 판단하기

# 1. 데이터 로드
해당 데이터는 네이버의 영화 리뷰 데이터를 모아놓은 것이다.

In [39]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

path_to_train_file = tf.keras.utils.get_file('train_txt', 'https://raw.githubusercontent.com/hmkim312/datas/main/navermoviereview/ratings_train.txt')
path_to_test_file = tf.keras.utils.get_file('test_txt', 'https://raw.githubusercontent.com/hmkim312/datas/main/navermoviereview/ratings_test.txt')

train_data = pd.read_csv(path_to_train_file, sep='\t')
test_data = pd.read_csv(path_to_test_file, sep='\t')

# 데이터 확인
train_data.head()


,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


# 2. 데이터 전처리

In [40]:
# Null 값 제거
train_data = train_data.dropna(how='any')
test_data = test_data.dropna(how='any')

# 특수 문자 제거 및 단어로 쪼개기
train_data['document'] = train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
train_data['document'] = [sentence.split(' ') for sentence in train_data['document']]
test_data['document'] = test_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]","")
test_data['document'] = [sentence.split(' ') for sentence in test_data['document']]

# 토큰화 (Text -> Sequence)
tokenizer = Tokenizer(num_words=20000)
tokenizer.fit_on_texts(train_data['document'])

X_train = tokenizer.texts_to_sequences(train_data['document'])
X_test = tokenizer.texts_to_sequences(test_data['document'])

y_train = np.array(train_data['label'])
y_test = np.array(test_data['label'])

마찬가지로 시퀀스 패딩 추가해주기

In [41]:
# 패딩
max_len = 25
X_train = pad_sequences(X_train,padding='post', maxlen=max_len)
X_test = pad_sequences(X_test,padding='post', maxlen=max_len)

# 3. 모델 생성
1. 임베딩 벡터 input_dim=20000, output_dim = 300, input_length=max_len
2. LSTM hidden layer 64개
3. Dense 출력 1개, activation은 sigmoid

In [42]:
nlp_model = Sequential([
    Embedding(input_dim=20000, output_dim=300, input_length=max_len),
    LSTM(64, return_sequences=False),
    Dense(1, activation='sigmoid')
])

nlp_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
nlp_model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 25, 300)           6000000   
                                                                 
 lstm_1 (LSTM)               (None, 64)                93440     
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 6093505 (23.24 MB)
Trainable params: 6093505 (23.24 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


# 4. 모델 학습

In [ ]:
nlp_history = nlp_model.fit(X_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
1875/1875 [==============================] - 256s 135ms/step - loss: 0.4802 - accuracy: 0.7466 - val_loss: 0.4301 - val_accuracy: 0.7860
Epoch 2/5
1875/1875 [==============================] - 250s 133ms/step - loss: 0.3728 - accuracy: 0.8103 - val_loss: 0.4378 - val_accuracy: 0.7831
Epoch 3/5
1875/1875 [==============================] - 254s 135ms/step - loss: 0.3182 - accuracy: 0.8335 - val_loss: 0.4987 - val_accuracy: 0.7772
Epoch 4/5
1875/1875 [==============================] - 251s 134ms/step - loss: 0.2771 - accuracy: 0.8511 - val_loss: 0.5231 - val_accuracy: 0.7740
Epoch 5/5
 458/1875 [======>.......................] - ETA: 3:06 - loss: 0.2312 - accuracy: 0.8747

# 5. 모델 평가 및 예측

In [44]:
test_loss, test_acc = nlp_model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc:.4f}')

1563/1563 [==============================] - 31s 20ms/step - loss: 0.5949 - accuracy: 0.7584
Test Accuracy: 0.7584


In [45]:
test_sentence = '지루할 줄 알았는데 너무 재밌었다'

test_sentence = test_sentence.split(' ')
test_sentences =[]
now_sentence = []
for word in test_sentence:
    now_sentence.append(word)
    test_sentences.append(now_sentence[:])

test_X_1 = tokenizer.texts_to_sequences(test_sentences)
test_X_1 = pad_sequences(test_X_1, padding = 'post', maxlen=max_len)
prediction = nlp_model.predict(test_X_1)
for idx, sentence in enumerate(test_sentences):
    print(sentence)
    print(prediction[idx])

1/1 [==============================] - 0s 475ms/step
['지루할']
[0.49168205]
['지루할', '줄']
[0.48779213]
['지루할', '줄', '알았는데']
[0.40793136]
['지루할', '줄', '알았는데', '너무']
[0.45090854]
['지루할', '줄', '알았는데', '너무', '재밌었다']
[0.9056931]


In [46]:
test_sentence = '괜찮을 줄 알았는데 지루하고 재미없었다.'

test_sentence = test_sentence.split(' ')
test_sentences =[]
now_sentence = []
for word in test_sentence:
    now_sentence.append(word)
    test_sentences.append(now_sentence[:])

test_X_1 = tokenizer.texts_to_sequences(test_sentences)
test_X_1 = pad_sequences(test_X_1, padding = 'post', maxlen=max_len)
prediction = nlp_model.predict(test_X_1)
for idx, sentence in enumerate(test_sentences):
    print(sentence)
    print(prediction[idx])

1/1 [==============================] - 0s 41ms/step
['괜찮을']
[0.7870225]
['괜찮을', '줄']
[0.6561334]
['괜찮을', '줄', '알았는데']
[0.38337335]
['괜찮을', '줄', '알았는데', '지루하고']
[0.15542373]
['괜찮을', '줄', '알았는데', '지루하고', '재미없었다.']
[0.00052879]
